<a href="https://colab.research.google.com/github/tutur90/RAG/blob/main/Copie_de_Project_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project : RAG

The goal of this project is to create a simple LLMs based RAG module. Obviously the most complex part is how to correctly link all components. We let you free to create your own database. Anything can be ragged, from research paper, songs, subtitles, books,...

This project is voluntarly sparsely helped, as, as engineers, you will dig into lots of existing method, and will need to pick the best one up. We want you to get familiar with the engineering world.

We will give you some recommendations. You will see lots of issues during this project, some you've already seen during Labs, other that are new. So buckle up, read the docs, and RAG your data.

Obviously there are lots of tutorials of the internet that you could just copy paste to get a baseline.


## **We encourage you to do code versioning using Github.**


**Ideal Project Timeline:**

*   Talking and getting to know the Modules. Discussing about the choice of your LLMs and the environment you'll have. Choosing a first set of data to RAG. Setting up your Github (Optional) (1h)
*   Setting up your first RAG Chain using Langchain or other (2-3h)
*   Understand the limitation of your RAG and find enhancements to set up your 2nd RAG. (2h)
*   Unveiling the unknown document, adapt your RAGs (2h)
*   Deploy it (Optional)
*   Begin your presentation (2h)
*   Presentation (5 min/groups)


We'll evaluate your presentation quality, your RAG system's capability, and your progress throughout the project sessions.

**Students who missed the first session will start with a score of 0 in the progress part**. :-)


Presentation:
- The number of slides you can do is unlimited
- You only have 5 min to present your project. We will stop you at 5 min whether you've finished or not
- Your presentation should include:
  - a presentation of your workflow (Agile Methodology, What's the job of each one...)
  - a presentation of your final pipeline with all enhancements done
  - a proof a work of your RAG on your data, and on the unknown data
  - what limitations you have and how to tackle them. For each too obvious limitations (more GPUs, more RAM..) : -1
  - if you've deployed your RAG, a scannable QR code to live test it.

#  Preliminaries : Some useful downloads

We give you some useful frameworks, that you could use to build your RAG.

In [7]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
# !%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00:00:0100:01


In [9]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

# I - Data Sourcing

Data Sourcing for RAG is really simple. Some questions to guide you:

* What data do we need ?
* How can we correctly parse the data ?
* Does the vector index provides a good representation for the vector database ?
* For example, using FAISS, can you easily retrieve your document ?

To begin, pick a simple story and store it in a Vector Database. You have the choice between multiple VectorStores (ChromaDB, QDrant,...)


In [10]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/kaggle/input/rag-test/")
data = loader.load()
len(data)

0

In [11]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

# II - Module Creation

Should you create a RAG Module, you need or not a LLM. We encourage you to test some LLMs (Mistral, LLama, Falcon, Gemma, ...) However, be aware that you won't have the space to run it on this colab.

We highly recommend to use LangChain, to build your Q&A app.

Some Questions to guide you:
* What model is easily accesible
* Are there any existing code to begin with ?
* What about the prompts ?
* What about the document parsing ?


In [12]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
modelPath='BAAI/bge-large-zh-v1.5'

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [13]:
db = FAISS.from_documents(docs, embeddings)

IndexError: list index out of range

In [ ]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

In [ ]:
# # Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
# tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# # Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
# model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

## Models

In [ ]:
from torch import bfloat16
import transformers

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=bfloat16,
    device_map='auto',
    
)
model.eval()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
pip = pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=False,  # if using langchain set True
    task="text-generation",
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # if output begins repeating increase
    do_sample=True,
)
llm = HuggingFacePipeline(
    pipeline=pip,
    )

In [ ]:
llm.invoke("What is cheesemaking?")

## Retrivers

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

docs = retriever.get_relevant_documents("What is Cheesemaking?")
print(docs[0].page_content) 

In [ ]:


# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents. 
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [ ]:
question = "Who is Laura?"
result = qa.invoke({"query": question})
print(result)

# III - Model Serving (Optional and for the best)

You can inspire yourself from the first hands-on, if your feeling powerful, you can build something using fastapi and push your deployed RAG into a simple github.io instance. Or just use the gradio deploiement framework.